### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 we can possibly look for jump operator of the form:

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or

$
A = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [2]:
@polyvar e[1]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [3]:
method = "T0-fix-Phi-up_"

@polyvar a[1:4]
@polyvar b[1:4]


#"just-zero-down_ "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2] + im*b[2] 
#          0                -a[1]-im*b[1]  ]


#"fix-Phi-up_: "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
#          a[3] + im*b[3]   -a[1]-im*b[1] ]


#"zero Temperature and fix-Phi-up_: "

Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
          0                -a[1]-im*b[1] ]

#"zero Temperature and fix-Phi-diag_: "

#Aˢʸᵐᵇ = [ a[1]     a[2]+im*b[2] 
#          0        -a[1]        ]




2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂
 0+0im           -a₁ + (0-1im)b₁

In [4]:
pwd()

"/home/zah/POP23"

In [5]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

test_files = ["State_D"*string(n) for n=1:20];

/home/zah/POP23/DATA/


### Train models loop

In [6]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP_trainB4_"*method*date_and_time_string * ".h5"
tests_data_file_name = "POP_testD20_"*method*date_and_time_string * ".h5"

println(models_file)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ

    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    
    start_time = time()
    
    objₑₓ = 0

    for (ρ,t) in zip([ρᵍ, ρᵉ, ρˣ, ρʸ], [tᵍ, tᵉ, tˣ, tʸ])
        
        t = convert(Vector{Float64}, t)
        @assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]

        # Convert cut ρ series:
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
        objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
        
    end # of files (initial states) loop
    
      
    solₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓ) 
    
    runtime = time() - start_time
    
    Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)
    Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)
    
    H = DenseOperator(basis, Hˢⁱᵈₑₓ)
    A = DenseOperator(basis, Aˢⁱᵈₑₓ)

    
    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)

        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈₑₓ)
        γ_group["A"] = convert.(ComplexF64, Aˢⁱᵈₑₓ)
        
        γ_group["runtime"] = runtime
        
    end # of HDF5 writing
    
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρ, t = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)

        ρᵗᵉˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρ]

        t = convert(Vector{Float64}, t)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(t, ρᵗᵉˢᵗ[1], H, [A])

        F = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗᵉˢᵗ, ρᴸᴹᴱ)]

        runtime = time() - start_time

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["fidelity"] = convert.(Float64, F)
            init_state_group["runtime"] = runtime
        end
                    
        print(minimum(F))
        print(" ")
    
    end
                
    println()

end

POP_trainB4_T0-fix-Phi-up_2023-Jul-29_at_20-10.h5
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0013068156581235518

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.000005%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0013067597563595557
Found a locally optimal solution by Ipopt, giving an upper bound: 0.01185694.
The relative 

State_D1 0.9991169363006824 State_D2 0.9995717648143699 State_D3 0.9991884421567065 State_D4 0.9995078788627128 State_D5 0.9994339599824615 State_D6 0.9993220223075939 State_D7 0.9977397132611057 State_D8 0.9988297628813839 State_D9 0.9993108835746162 State_D10 0.9994235544764112 State_D11 0.9990274862990678 State_D12 0.9994338243308778 State_D13 0.9984447482722294 State_D14 0.9989943976770921 State_D15 0.9996456852551345 State_D16 0.9996297829301102 State_D17 0.9974042859261141 State_D18 0.9991324117316933 State_D19 0.9994394207875955 State_D20 0.9988099094249738 
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.015071847987465747
Global optimality certified with relative optimality gap 0.000831%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is la

In [7]:
tests_data_file_name

"POP_testD20_T0-fix-Phi-up_2023-Jul-29_at_20-10.h5"